<a href="https://colab.research.google.com/github/grey4jtb/Assignment3/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy
#first install the library that would help us use BERT in an easy to use interface
#https://github.com/UKPLab/sentence-transformers/tree/master/sentence_transformers
!pip install -U sentence-transformers
!pip install -U bert-extractive-summarizer
!python -m spacy download en_core_web_sm

In [ ]:
## Use this to import spacy directly if using the Brain colab runtime or a custom colab runtime that includes spacy in its build.
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from summarizer.sbert import SBertSummarizer
from sentence_transformers import SentenceTransformer, util
from string import punctuation
from heapq import nlargest
import os
import re
import spacy
from spacy import displacy
import pandas as pd
import scipy.spatial
import pickle as pkl
import pandas as pd
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")

In [ ]:
stopwords=list(STOP_WORDS)
punctuation=punctuation+ '\n'
embedder = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
! pip install kaggle

In [9]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d hamzafarooq50/hotel-listings-and-reviews

 59% 5.00M/8.54M [00:00<00:00, 32.9MB/s]
100% 8.54M/8.54M [00:00<00:00, 41.9MB/s]


In [11]:
! unzip /content/hotel-listings-and-reviews.zip

Archive:  /content/hotel-listings-and-reviews.zip
  inflating: HotelListInAthens__en2019100120191005.csv  
  inflating: HotelListInAustin__en2019100120191005.csv  
  inflating: HotelListInBali__en2019100120191005.csv  
  inflating: HotelListInBangkok__en2019100120191005.csv  
  inflating: HotelListInBarcelona__en2019100120191005.csv  
  inflating: HotelListInBombay__en2019100120191005.csv  
  inflating: HotelListInChicago__en2019100120191005.csv  
  inflating: HotelListInDubai__en2019100120191005.csv  
  inflating: HotelListInHong Kong__en2019100120191005.csv  
  inflating: HotelListInIstanbul__en2019100120191005.csv  
  inflating: HotelListInLondon__en2019100120191005.csv  
  inflating: HotelListInMiami__en2019100120191005.csv  
  inflating: HotelListInMilan__en2019100120191005.csv  
  inflating: HotelListInNew York__en2019100120191005.csv  
  inflating: HotelListInOsaka__en2019100120191005.csv  
  inflating: HotelListInParis__en2019100120191005.csv  
  inflating: HotelListInPhuket__e

In [12]:
df = pd.read_csv("/content/hotelReviewsInMiami__en2019100120191005.csv")

In [13]:
df = df[['review_body','hotelName']]
df

,review_body,hotelName
0,Great hotel to party over the 4thj of July wee...,0 Eden Roc Miami Beach Resort\n2 Eden Ro...
1,Awesome place to stay at the beach. Great hote...,0 Eden Roc Miami Beach Resort\n2 Eden Ro...
2,There are few words to accurately depict Marth...,0 Eden Roc Miami Beach Resort\n2 Eden Ro...
3,Danni at the pool is awesome! Her customer ser...,0 Eden Roc Miami Beach Resort\n2 Eden Ro...
4,Know that this is an expensive hotel. It is a ...,0 Eden Roc Miami Beach Resort\n2 Eden Ro...
...,...,...
1795,"Amazing hotel, staff outstanding and friendly....","31 The Setai, Miami Beach\nName: hotel_name..."
1796,The non-descript hotel front belies the luxury...,"31 The Setai, Miami Beach\nName: hotel_name..."
1797,The Setai is incredible and I will never stay ...,"31 The Setai, Miami Beach\nName: hotel_name..."
1798,We are staying at the Setai in Miami at the mo...,"31 The Setai, Miami Beach\nName: hotel_name..."


In [14]:
df_miami = df.sort_values(['hotelName']).groupby('hotelName', sort=False).review_body.apply(''.join).reset_index(name='review_body')

In [15]:
summ_model = SBertSummarizer()
def summarized_review(data):
  model = SBertSummarizer('paraphrase-MiniLM-L6-v2')
  result = model(data, num_sentences=3)
  return result

df_summary = df_miami.copy()
df_summary['summary'] = df_miami['review_body'].apply(summarized_review)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
df_miami['summary'] = df_summary['summary']
df_miami.head(3)

,hotelName,review_body,summary
0,0 Eden Roc Miami Beach Resort\n2 Eden Ro...,Great hotel to party over the 4thj of July wee...,Great hotel to party over the 4thj of July wee...
1,"1 Fontainebleau\nName: hotel_name, dtype: o...",We enjoyed excellent beach service brought to ...,We enjoyed excellent beach service brought to ...
2,"10 Marriott Vacation Club Pulse, South Beac...",After getting off the Carnival ship at 8:30 a....,After getting off the Carnival ship at 8:30 a....


In [17]:
df_miami['review_body'] = df_miami['review_body'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str
df_miami['review_body']= df_miami['review_body'].apply(lambda x: lower_case(x))


In [18]:
df = df_miami
df_sentences = df_miami.set_index("review_body")
df_sentences = df_sentences["hotelName"].to_dict()
df_sentences_list = list(df_sentences.keys())
len(df_sentences_list)

31

In [20]:
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]

100%|██████████| 31/31 [00:00<00:00, 15942.06it/s]


In [21]:
# Corpus with example sentences
corpus = df_sentences_list
corpus_embeddings = embedder.encode(corpus,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
queries = ['Hotel closest to beach',
           'hotel with excellent breakfast']
model = SentenceTransformer('all-MiniLM-L6-v2')
paraphrases = util.paraphrase_mining(model, corpus)
query_embeddings = embedder.encode(queries,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 3
print("\nTop 3 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n=========================================================")
    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        hotel = df.loc[df['review_body']== corpus[idx]]['hotelName']
        summary = df.loc[df['review_body']== corpus[idx]]['summary']
        print("Hotel Name:  " , hotel , "\n")
        print("Summary:  " , summary , "\n")
        print("-------------------------------------------")


Top 3 most similar sentences in corpus:


==========================Query==============================
=== Hotel closest to beach =====
Score:    (Score: 0.6948) 

Hotel Name:   12    20    Red South Beach Hotel\nName: hotel_name,...
Name: hotelName, dtype: object 

Summary:   12    The hotel is right on the beach (10 mins walk)...
Name: summary, dtype: object 

-------------------------------------------
Score:    (Score: 0.6900) 

Hotel Name:   20    28    Urbanica The Meridian Hotel\nName: hotel...
Name: hotelName, dtype: object 

Summary:   20    I staid at the hotel last weekend and it was t...
Name: summary, dtype: object 

-------------------------------------------
Score:    (Score: 0.6548) 

Hotel Name:   4    12    Crowne Plaza South Beach - Z Ocean Hotel...
Name: hotelName, dtype: object 

Summary:   4    The hotel is about 25 minutes away from the ai...
Name: summary, dtype: object 

-------------------------------------------


==========================Query============